In [1]:
import pandas as pd
import os

# os.environ['PYSPARK_SUBMIT_ARGS'] = "--master mymaster --total-executor 2 --conf 'spark.driver.extraJavaOptions=-Dhttp.proxyHost=proxy.mycorp.com-Dhttp.proxyPort=1234' -Dhttp.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 -Dhttps.proxyHost=proxy.mycorp.com -Dhttps.proxyPort=1234 -Dhttps.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 pyspark-shell"
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, isnan, when, count
from pyspark.ml.feature import Imputer, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import os

from DataLoader import DataLoader, DataSet


In [2]:
loader = DataLoader(data_set=DataSet.VALIDATION)
train_df = loader.df

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 14:50:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/11 14:51:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/train-8.csv
24/03/11 14:51:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id,

CodeCache: size=131072Kb used=22717Kb max_used=22717Kb free=108354Kb
 bounds [0x00000001091e0000, 0x000000010a830000, 0x00000001111e0000]
 total_blobs=8951 nmethods=8022 adapters=842
 compilation: disabled (not enough contiguous free space left)
Removed 0 duplicate rows.


In [3]:
# Count missing values in each column
missing_value_counts = train_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train_df.columns])

# Display the count of missing values for each column
missing_value_counts.show()

24/03/11 14:51:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/train-8.csv
24/03/11 14:51:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/

+---+----------+--------------+-------------+----+-----------------+---------------+-----------+-----------+--------------+-------------------+-----+
| id|product_id|product_parent|product_title|vine|verified_purchase|review_headline|review_body|review_date|marketplace_id|product_category_id|label|
+---+----------+--------------+-------------+----+-----------------+---------------+-----------+-----------+--------------+-------------------+-----+
|  0|         0|             0|           64|   0|                0|            636|          0|          1|             0|                  0|    0|
+---+----------+--------------+-------------+----+-----------------+---------------+-----------+-----------+--------------+-------------------+-----+



In [4]:
train_df.groupBy("vine").count().show()

+----+-----+
|vine|count|
+----+-----+
|   Y|   14|
|   N| 8549|
+----+-----+



In [5]:
train_df.groupBy("verified_purchase").count().show()

+-----------------+-----+
|verified_purchase|count|
+-----------------+-----+
|                Y| 5953|
|                N| 2610|
+-----------------+-----+



In [6]:
train_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|False| 4991|
| True| 3572|
+-----+-----+

